In [1]:
import numpy as np 
import pandas as pd

In [6]:
import os
for dirname, _, filenames in os.walk('/content/FD'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/FD/Training Data.csv
/content/FD/click_log.csv
/content/FD/Test Data.csv
/content/FD/Pre-Read.txt
/content/FD/sample_submission.csv


In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn import linear_model
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
data = pd.read_csv('/content/FD/Training Data.csv')

In [10]:
data.head()

clientid_cr  ... conversion_fraud
0        75694  ...             True
1        75694  ...             True
2        75694  ...             True
3        75694  ...             True
4        30431  ...             True

[5 rows x 56 columns]

In [11]:
data.dtypes

clientid_cr                        int64
clmbuserid_cr                     object
conversiontime_cr                float64
imprid_cr                         object
adslotdimid_cr                     int64
algo_cr                          float64
audiences_cr                      object
clickbid_cr                      float64
geodimid_cr                        int64
ip_cr                             object
itemcolumbiaid_cr                  int64
itemid_cr                          int64
position_cr                      float64
pubclientid_cr                     int64
refurl_cr                         object
siteId_cr                          int64
templateid_cr                    float64
goalid_cr                          int64
time_cr                          float64
adLogType_cr                       int64
v_cr                             float64
allAudiences_cr                   object
pricingtype_cr                     int64
osId_cr                          float64
browserId_cr    

In [12]:
data['conversion_fraud'] = data['conversion_fraud'].astype(int)
data

clientid_cr  ... conversion_fraud
0          75694  ...                1
1          75694  ...                1
2          75694  ...                1
3          75694  ...                1
4          30431  ...                1
..           ...  ...              ...
960         2139  ...                0
961        70491  ...                0
962        85572  ...                0
963        75694  ...                0
964        75694  ...                0

[965 rows x 56 columns]

In [13]:
data.shape
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 965 entries, 0 to 964
Data columns (total 56 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   clientid_cr                    965 non-null    int64  
 1   clmbuserid_cr                  965 non-null    object 
 2   conversiontime_cr              965 non-null    float64
 3   imprid_cr                      965 non-null    object 
 4   adslotdimid_cr                 965 non-null    int64  
 5   algo_cr                        945 non-null    float64
 6   audiences_cr                   766 non-null    object 
 7   clickbid_cr                    669 non-null    float64
 8   geodimid_cr                    965 non-null    int64  
 9   ip_cr                          945 non-null    object 
 10  itemcolumbiaid_cr              965 non-null    int64  
 11  itemid_cr                      965 non-null    int64  
 12  position_cr                    945 non-null    flo

In [14]:
data.describe()

clientid_cr  conversiontime_cr  ...   bundleId_cr  conversion_fraud
count    965.000000       9.650000e+02  ...  5.000000e+00        965.000000
mean   62726.958549       1.620000e+12  ...  5.326108e+07          0.135751
std    25456.725592       0.000000e+00  ...  9.673626e+05          0.342702
min      802.000000       1.620000e+12  ...  5.181866e+07          0.000000
25%    64588.000000       1.620000e+12  ...  5.269602e+07          0.000000
50%    75694.000000       1.620000e+12  ...  5.393023e+07          0.000000
75%    75694.000000       1.620000e+12  ...  5.393023e+07          0.000000
max    86555.000000       1.620000e+12  ...  5.393025e+07          1.000000

[8 rows x 44 columns]

In [15]:
df = data.select_dtypes(exclude=['float64','object'])
df

clientid_cr  adslotdimid_cr  ...  goalTypeId_cr  conversion_fraud
0          75694          321300  ...             10                 1
1          75694          129239  ...             10                 1
2          75694          321554  ...             10                 1
3          75694          321300  ...             10                 1
4          30431          355768  ...             10                 1
..           ...             ...  ...            ...               ...
960         2139          321114  ...             10                 0
961        70491          334758  ...             10                 0
962        85572          348089  ...             10                 0
963        75694          356952  ...             10                 0
964        75694          356952  ...             10                 0

[965 rows x 13 columns]

In [16]:
df.corr()

clientid_cr  adslotdimid_cr  ...  goalTypeId_cr  conversion_fraud
clientid_cr           1.000000       -0.104231  ...            NaN          0.088796
adslotdimid_cr       -0.104231        1.000000  ...            NaN          0.021717
geodimid_cr           0.018064        0.017857  ...            NaN         -0.012778
itemcolumbiaid_cr    -0.244687        0.055494  ...            NaN         -0.033403
itemid_cr             0.052119        0.083087  ...            NaN          0.014677
pubclientid_cr        0.043173        0.327938  ...            NaN         -0.117970
siteId_cr            -0.123359        0.542110  ...            NaN         -0.123216
goalid_cr             0.665659       -0.043798  ...            NaN          0.043013
adLogType_cr          0.314901        0.038599  ...            NaN          0.122550
pricingtype_cr       -0.045195        0.011045  ...            NaN          0.036736
lookUpFrom_cr         0.207429        0.035138  ...            NaN          0.105829
goalTypeId_cr              NaN             NaN  ...            NaN               NaN
conversion_fraud      0.088796        0.021717  ...            NaN          1.000000

[13 rows x 13 columns]

In [17]:
df_1 = df.drop(['goalTypeId_cr','itemcolumbiaid_cr','pubclientid_cr','siteId_cr','geodimid_cr'], axis = 1)
df_1.corr()

clientid_cr  adslotdimid_cr  ...  lookUpFrom_cr  conversion_fraud
clientid_cr          1.000000       -0.104231  ...       0.207429          0.088796
adslotdimid_cr      -0.104231        1.000000  ...       0.035138          0.021717
itemid_cr            0.052119        0.083087  ...      -0.131437          0.014677
goalid_cr            0.665659       -0.043798  ...       0.131741          0.043013
adLogType_cr         0.314901        0.038599  ...       0.910270          0.122550
pricingtype_cr      -0.045195        0.011045  ...       0.133794          0.036736
lookUpFrom_cr        0.207429        0.035138  ...       1.000000          0.105829
conversion_fraud     0.088796        0.021717  ...       0.105829          1.000000

[8 rows x 8 columns]

In [18]:
df_2 = pd.DataFrame(df_1,columns=['clientid_cr','adslotdimid_cr','itemid_cr','goalid_cr','adLogType_cr','pricingtype_cr','lookUpFrom_cr','conversion_fraud'])
df_2

clientid_cr  adslotdimid_cr  ...  lookUpFrom_cr  conversion_fraud
0          75694          321300  ...              2                 1
1          75694          129239  ...              2                 1
2          75694          321554  ...              2                 1
3          75694          321300  ...              2                 1
4          30431          355768  ...              2                 1
..           ...             ...  ...            ...               ...
960         2139          321114  ...              2                 0
961        70491          334758  ...              2                 0
962        85572          348089  ...              2                 0
963        75694          356952  ...              2                 0
964        75694          356952  ...              2                 0

[965 rows x 8 columns]

In [19]:
X = df_2[['clientid_cr','adslotdimid_cr','itemid_cr','goalid_cr','adLogType_cr','pricingtype_cr','lookUpFrom_cr']] 
Y = df_2['conversion_fraud']
regr = linear_model.LinearRegression()
regr.fit(X, Y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
X = sm.add_constant(X)  
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
print_model = model.summary()
print(print_model)

Intercept: 
 0.29313503916640654
Coefficients: 
 [ 4.52275917e-07  1.23652771e-07 -8.77107102e-09 -1.26592423e-07
  3.95722690e-01  3.31303566e-03 -3.02229936e-01]
                            OLS Regression Results                            
Dep. Variable:       conversion_fraud   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     2.983
Date:                Mon, 07 Jun 2021   Prob (F-statistic):            0.00422
Time:                        16:24:49   Log-Likelihood:                -324.95
No. Observations:                 965   AIC:                             665.9
Df Residuals:                     957   BIC:                             704.9
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                     coef    std err          